### Full algorithm
HCHS_data(cleaning) -> replace abbrviations -> translation with Hugging Face -> rid of unnecessory information

#### HCHS data

In [1]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
from transformers import pipeline

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-04-13 13:03:10.127659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-13 13:03:10.127791: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
HCHS_data=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/VM_Soarian_HCHS_20210422.xlsx')
HCHS_data['Langname']=HCHS_data['Langname'].replace([' ', ''], np.nan)
HCHS_Langname=HCHS_data['Langname'].str.strip()
HCHS_Langname=HCHS_Langname.dropna(axis=0)
HCHS_Langname=HCHS_Langname.unique()

In [3]:
df=pd.DataFrame()
df['HCHS_German']=HCHS_Langname

In [4]:
df['HCHS_German']=df['HCHS_German'].str.replace(r'[{4}|]', ',', regex=True)#cases with |

#### Abbriviation replacement

In [6]:
list_abbr=pd.read_excel('/workspaces/de.uke.iam.automapping/experiments/german_abbreviation.xlsx')

In [7]:
df['HCHS_German_without_abbreviation']=df['HCHS_German']

In [8]:
#abbreviation replacment
for i in tqdm(range(list_abbr.shape[0])):
    abbreviation=list_abbr['Abbreviation'][i]
    description=list_abbr['Description'][i]
    df['HCHS_German_without_abbreviation']=df['HCHS_German_without_abbreviation'].str.replace(r'\b' + abbreviation + r'[^\w]' ,description+' ',  regex= True)

100%|██████████| 874/874 [00:03<00:00, 219.94it/s]


#### Translation German-English

In [9]:
model="Helsinki-NLP/opus-mt-de-en"
translator=pipeline('translation', model)

In [10]:
tr=translator(df['HCHS_German_without_abbreviation'].to_list())

In [11]:
translation_list=[]
for i in tr:
    translation_list.append(i.get('translation_text'))

In [12]:
df['HCHS_English']=translation_list

#### Rid of unnecessory information

In [14]:
nlp=spacy.load('en_core_web_lg')

In [15]:
english_ready=[]
for sentence in df['HCHS_English'].to_list():
    token_list=[]
    doc=nlp(sentence)
    token_list=[str(token) for token in doc if not token.is_stop and not token.is_punct and str(token)!='doctor']
    text = " ".join(token_list)
    english_ready.append(text)

In [16]:
df['HCHS_new']=english_ready

In [17]:
df

,HCHS_German,HCHS_German_without_abbreviation,HCHS_English,HCHS_new
0,Liegt bei Ihnen eine vom Arzt diagnostizierte ...,Liegt bei Ihnen eine vom Arzt diagnostizierte ...,Do you have dementia diagnosed by the doctor?,dementia diagnosed
1,Liegt bei Ihnen ein vom Arzt diagnostizierter ...,Liegt bei Ihnen ein vom Arzt diagnostizierter ...,Is there a diabetes diagnosed by your doctor?,diabetes diagnosed
2,Wann war die letzte Injektion?,Wann war die letzte Injektion?,When was the last injection?,injection
3,Hatten Sie bereits einen vom Arzt diagnostizie...,Hatten Sie bereits einen vom Arzt diagnostizie...,Have you already had a heart attack diagnosed ...,heart attack diagnosed
4,Ist bei Ihnen eine vom Arzt diagnostizierte He...,Ist bei Ihnen eine vom Arzt diagnostizierte He...,Are you aware of a heart failure diagnosed by ...,aware heart failure diagnosed
...,...,...,...,...
948,Kreuzbiss,Kreuzbiss,Crossbite,Crossbite
949,Mundöffnungsbewegung,Mundöffnungsbewegung,Mouth opening movement,Mouth opening movement
950,max. aktive Mundöffnung,max. aktive Mundöffnung,max. active mouth opening,max active mouth opening
951,max. passive Mundöffnung,max. passive Mundöffnung,max. passive mouth opening,max passive mouth opening


In [18]:
df.to_excel('HCHS_full_cycle.xlsx', encoding='utf-16')